In [1]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
# Read csv into DataFrame
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop ID columns
application_df.drop(columns=["EIN","NAME"],inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Count the unique values in each column
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Value counts for APPLICATION_TYPE column
app_type_counts = application_df["APPLICATION_TYPE"].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# List of application types to replace with "Other"
application_types_to_replace = app_type_counts.loc[lambda x: x < 500].index
application_types_to_replace

Index(['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object', name='APPLICATION_TYPE')

In [7]:
# Replace
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app,"Other")

# Check if replacement was successful
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Value counts for CLASSIFICATION column
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# List of classifications to replace with "Other"
classifications_to_replace = classification_counts.loc[lambda x: x < 1000].index
classifications_to_replace

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278',
       'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600',
       'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256',
       'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820',
       'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570',
       'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561',
       'C4500', 'C2150'],
      dtype='object', name='CLASSIFICATION')

In [10]:
# Replace
for classif in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(classif,"Other")

# Check if replacement was successful
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric
numeric_df = pd.get_dummies(application_df)
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   IS_SUCCESSFUL                 34299 non-null  int64
 3   APPLICATION_TYPE_Other        34299 non-null  bool 
 4   APPLICATION_TYPE_T10          34299 non-null  bool 
 5   APPLICATION_TYPE_T19          34299 non-null  bool 
 6   APPLICATION_TYPE_T3           34299 non-null  bool 
 7   APPLICATION_TYPE_T4           34299 non-null  bool 
 8   APPLICATION_TYPE_T5           34299 non-null  bool 
 9   APPLICATION_TYPE_T6           34299 non-null  bool 
 10  APPLICATION_TYPE_T7           34299 non-null  bool 
 11  APPLICATION_TYPE_T8           34299 non-null  bool 
 12  AFFILIATION_CompanySponsored  34299 non-null  bool 
 13  AFFILIATION_Family/Parent     3

In [12]:
# Split dataframe into features and target arrays
y = numeric_df["IS_SUCCESSFUL"]
X = numeric_df.drop(columns="IS_SUCCESSFUL")

# Split features and target into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)

In [13]:
# Instantiate StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training features
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=43,activation="relu",input_dim=43))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=20,activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1,activation="tanh"))

# Check model structure
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 43)                1892      
                                                                 
 dense_1 (Dense)             (None, 20)                880       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2793 (10.91 KB)
Trainable params: 2793 (10.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-07-30 10:40:44.992646: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-07-30 10:40:44.992665: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-07-30 10:40:44.992669: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-07-30 10:40:44.992696: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-30 10:40:44.992709: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# Compile the model
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [16]:
# Train the model
model.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25
  1/804 [..............................] - ETA: 4:08 - loss: 4.1421 - accuracy: 0.6562

2023-07-30 10:40:45.263649: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 4s 5ms/step - loss: 1.6284 - accuracy: 0.6680
Epoch 2/25
804/804 [==============================] - 4s 5ms/step - loss: 1.1491 - accuracy: 0.6961
Epoch 3/25
804/804 [==============================] - 4s 5ms/step - loss: 1.1116 - accuracy: 0.6976
Epoch 4/25
804/804 [==============================] - 4s 5ms/step - loss: 1.1127 - accuracy: 0.6939
Epoch 5/25
804/804 [==============================] - 4s 5ms/step - loss: 1.1251 - accuracy: 0.6879
Epoch 6/25
804/804 [==============================] - 4s 5ms/step - loss: 1.8348 - accuracy: 0.6507
Epoch 7/25
804/804 [==============================] - 4s 5ms/step - loss: 0.7890 - accuracy: 0.6812
Epoch 8/25
804/804 [==============================] - 4s 5ms/step - loss: 1.2328 - accuracy: 0.6793
Epoch 9/25
804/804 [==============================] - 4s 5ms/step - loss: 1.0852 - accuracy: 0.6855
Epoch 10/25
804/804 [==============================] - 4s 5ms/step - loss: 1.7460 - accuracy: 0.6274
Epoch 11/2

In [17]:
# Evaluate the model using test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2023-07-30 10:42:20.912056: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 4.9541 - accuracy: 0.6311 - 736ms/epoch - 3ms/step
Loss: 4.954089641571045, Accuracy: 0.6311370134353638


In [18]:
# Export the model
model.save("model1.h5")

/Users/vinnyshankar/anaconda3/envs/p4/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
# Define the model
model2 = tf.keras.models.Sequential()

# First hidden layer
model2.add(tf.keras.layers.Dense(units=80,activation="relu",input_dim=43))

# Second hidden layer
model2.add(tf.keras.layers.Dense(units=40,activation="sigmoid"))

# Output layer
model2.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Check model structure
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 40)                3240      
                                                                 
 dense_5 (Dense)             (None, 1)                 41        
                                                                 
Total params: 6801 (26.57 KB)
Trainable params: 6801 (26.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
model2.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [21]:
# Train the model
model2.fit(X_train_scaled,y_train,epochs=25)

Epoch 1/25


2023-07-30 10:48:13.783333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 4s 5ms/step - loss: 0.5790 - accuracy: 0.7185
Epoch 2/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5625 - accuracy: 0.7298
Epoch 3/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5567 - accuracy: 0.7303
Epoch 4/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5537 - accuracy: 0.7310
Epoch 5/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5524 - accuracy: 0.7315
Epoch 6/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5510 - accuracy: 0.7316
Epoch 7/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5499 - accuracy: 0.7331
Epoch 8/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5486 - accuracy: 0.7326
Epoch 9/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5481 - accuracy: 0.7339
Epoch 10/25
804/804 [==============================] - 4s 5ms/step - loss: 0.5471 - accuracy: 0.7341
Epoch 11/2

In [22]:
# Evaluate the model using test data
model_loss, model_accuracy = model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2023-07-30 10:49:46.242529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5506 - accuracy: 0.7266 - 713ms/epoch - 3ms/step
Loss: 0.5505609512329102, Accuracy: 0.7266472578048706


In [23]:
# Export the model
model2.save("model2.h5")

/Users/vinnyshankar/anaconda3/envs/p4/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
